# KNN with Clustering

The K Nearest Neighbors approach for recommendation usually works by, given a certain user, finding similar ones and analyzing their iteraction history. However, since we do not have a `user_id` in the dataset we used a workaround. With *clusters*, we can find the most itnteracted items for each cluster instead of each user. Whenever we wish to find similar users, the KNN algorithm can provide similar *feature combinations* and we can match those to specific clusters. The iteraction list can then be used to create scores for each item. Therefore, we'll need to save through BentoML the following data:

* An **Index Map** to map an item_id into an index (e.g. 1, 2, 7, 45, etc.)
* A **Clustering Algorithm** to map new user features to clusters
* A **Popularity Matrix** holding the amount of iteractions per cluster per item with shape (clusters, items)
* A **KNN Algorithm** to provide the K nearest neighbors of a set of `user_features`

In this notebook we will setup these elements. However, the actual recommendation happens in `knn_cluster.py` that will answer to the BentoML api when requested.

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

from preprocessing import preprocess
from knn_cluster import ClusteredKNN

### Acquire preprocessed Data

In [2]:
df = preprocess("Sample")

In [3]:
df.head(6)

,Timestamp,Clicked_Article,Click,User_Features,Article_List
0,1317513291,560620,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029..."
1,1317513291,565648,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029..."
2,1317513291,563115,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029..."
3,1317513292,552077,0,"[True, False, False, False, False, False, True...","[552077, 555224, 555528, 559744, 559855, 56029..."
4,1317513292,564335,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029..."
5,1317513292,565589,0,"[True, False, False, False, True, False, False...","[552077, 555224, 555528, 559744, 559855, 56029..."


## Clustering

For the cluster, we will need the users' features

In [4]:
users = np.asarray(df.loc[:,'User_Features']) # acquire only the features
users = np.stack(users, axis=0) # stack them to make an array (iteractions, features)
users.shape

(10447, 136)

Now we can intialize the clustering algorithm, decide how many clusters we want, and compute

In [5]:
N_CLUSTERS = 20
kmeans = KMeans(n_clusters = N_CLUSTERS)
kmeans.fit(users)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=20, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

## K Nearest Neighbors

For the KNN algorithm, we will need the users' features, as well as their clusters, referred here as `labels`

In [6]:
labels = kmeans.labels_
knn = KNeighborsClassifier()
knn.fit(users, labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

The KNN Algorithm will return a list of indexes of the k closest elements

First, we take a sample from the dataset

In [7]:
sample = df.sample(1).loc[:,'User_Features']
sample_features = np.stack(sample,axis=0)

Now we can run the KNN with the features sampled

In [8]:
idxs = knn.kneighbors(X=sample_features, n_neighbors=7, return_distance=False)
idxs

array([[6471, 2584, 7027, 6046,  521, 1184,  786]])

## Index Map

First, we get all articles in a list

In [9]:
articles = df['Clicked_Article'].unique()

Then, we iterate over them creating a dictionary for the index map.

In [10]:
index_map = {}
idx = 1 # idx starts at 1 so that 0 is used for when the article is not found in the index map
for art in articles:
    index_map[art] = idx
    idx+=1
# index_map

## Popularity Matrix

In order to get our Popularity Matrix, we need to replace the `article_id` by the index and add to which cluster an iteraction belongs to

In [11]:
df['Clicked_Article'].replace(index_map, inplace=True)
df['Cluster'] = kmeans.labels_
df.head(5)

,Timestamp,Clicked_Article,Click,User_Features,Article_List,Cluster
0,1317513291,1,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029...",13
1,1317513291,2,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029...",8
2,1317513291,3,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029...",0
3,1317513292,4,0,"[True, False, False, False, False, False, True...","[552077, 555224, 555528, 559744, 559855, 56029...",16
4,1317513292,5,0,"[True, False, False, False, False, False, Fals...","[552077, 555224, 555528, 559744, 559855, 56029...",1


Now we can locate the iteractions with clicks and group them by cluster and item.

In [12]:
popular_by_cluster = df.loc[(df['Click']==1)].groupby(['Cluster','Clicked_Article']).size().sort_values(ascending=False)
popular_by_cluster

Cluster  Clicked_Article
1        23                 11
         3                  10
         19                  9
         14                  9
         25                  9
                            ..
11       12                  1
10       23                  1
         22                  1
         16                  1
0        1                   1
Length: 193, dtype: int64

As this returns us with a pandas Series type, we need to create a matrix (numpy array) to hold these values. 

In [13]:
pop_matrix = np.zeros((N_CLUSTERS,len(index_map)+1), dtype='int')

for c in range(N_CLUSTERS):
#     pop_matrix[c][0] = -1
    for i in popular_by_cluster[c].index:
        pop_matrix[c][i] = popular_by_cluster[c][i]

pop_matrix

array([[ 0,  1,  1,  4,  0,  2,  2,  1,  1,  2,  1,  0,  1,  1,  1,  0,
         1,  0,  1,  0,  1,  1,  0,  0,  0,  1,  0],
       [ 0,  2,  2, 10,  2,  3,  7,  5,  5,  6,  8,  3,  5,  3,  9,  2,
         2,  6,  7,  9,  6,  5,  7, 11,  6,  9,  7],
       [ 0,  0,  1,  0,  1,  0,  0,  0,  0,  0,  2,  0,  0,  1,  0,  0,
         1,  0,  0,  2,  0,  0,  1,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,
         0,  0,  0,  1,  2,  0,  0,  1,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  3,  0,  1,  0,  0,  0,
         1,  0,  0,  0,  0,  0,  2,  1,  0,  1,  2],
       [ 0,  0,  0,  0,  0,  1,  0,  1,  1,  2,  1,  0,  0,  1,  0,  0,
         0,  1,  0,  0,  1,  0,  2,  0,  0,  1,  0],
       [ 0,  0,  0,  1,  0,  0,  1,  1,  0,  1,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  1,  1,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  1,  2,  0,  0,  0,  1,  1,  0,  1,  0,  0,
         0,  0,  1,  0,  1,  0,  1,  1,  0,  3,  0],


### Saving Artifacts

Now that we have all our elements to make recommendations, we can save them so our recommender can load them.

In [14]:
model = ClusteredKNN()

In [15]:
model.pack("index_map", index_map)

In [16]:
model.pack("pop_matrix", pop_matrix)

In [17]:
model.pack("knn", knn)

In [18]:
model.pack("cluster_path", kmeans)

After packing what our recommender will need, we can test it with a small sample.

Notice that, in this case, we must pass the number of neighbors to be used as `model.rank(sample, n_neighbors)`

In [19]:
test_articles = [565648, 563115, 552077, 564335, 565589, 563938, 560290, 563643, 560620, 565822, 563787, 555528, 565364, 559855, 560518]

In [20]:
model.rank({'Timestamp': 123456789, 'Clicked_Article': 565822, 'Click': 1, 'User_Features': sample_features[0], 'Article_List': np.asarray(test_articles)}, n_neighbors=10)

[565648,
 565589,
 563115,
 563643,
 565822,
 555528,
 564335,
 563938,
 565364,
 560290,
 552077,
 563787,
 560620,
 560518,
 559855]

In order to check wether the recommendation is correct, we can do it ourselves

First, we get the neighbors for our feature set

In [21]:
idxs = knn.kneighbors(X=sample_features, n_neighbors=10, return_distance=False)
idxs

array([[6471, 2584, 7027, 6046,  521, 1393, 4432, 1184, 6247, 2001]])

Then, we get the clusters for each of these neighbors

In [22]:
knclusters = kmeans.labels_[idxs]
knclusters

array([[11, 11, 11,  5, 11,  9, 11,  5, 11,  5]], dtype=int32)

It is likely that most neighbors will belong to the same cluster, but it should not always be the case.

Now we can look at our popularity matrix and gather the clicks for each cluster

In [23]:
clicks = [pop_matrix[c] for c in knclusters]
clicks = np.asarray(clicks[0])
clicks

array([[0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 3, 0, 1,
        0, 5, 0, 0, 0],
       [0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 3, 0, 1,
        0, 5, 0, 0, 0],
       [0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 3, 0, 1,
        0, 5, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        2, 0, 0, 1, 0],
       [0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 3, 0, 1,
        0, 5, 0, 0, 0],
       [0, 0, 2, 0, 1, 1, 0, 1, 0, 3, 3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0,
        0, 1, 0, 1, 3],
       [0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 3, 0, 1,
        0, 5, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        2, 0, 0, 1, 0],
       [0, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 0, 3, 0, 1,
        0, 5, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 1, 1, 2, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
        2, 0, 0, 1, 0]])

It is intended that clusters that appear more than once have their information added multiple times.

This is because we will mean over these values. Therefore, the clusters that appear multiple times should be weighted differently.

In [24]:
mean_scores = np.mean(clicks, axis=0)
mean_scores

array([0. , 0. , 1.4, 1.2, 0.1, 0.4, 1.2, 0.4, 0.3, 0.9, 0.6, 0. , 0.6,
       0.3, 0. , 0. , 0. , 1.7, 0. , 1.8, 0.3, 0.6, 0.6, 3.1, 0. , 0.4,
       0.3])

To get the scores specifically about the items in the recommended list, we need to translate the ids into indexes and look at those in the `mean_scores`

In [25]:
indexes = [index_map[art] for art in test_articles]
scores = [mean_scores[idx] for idx in indexes] 
scores

[1.4, 1.2, 0.1, 0.4, 1.2, 0.4, 0.3, 0.9, 0.0, 0.6, 0.0, 0.6, 0.3, 0.0, 0.0]

Sorting out the values we can check if the model functioned properly

In [26]:
sorted(zip(scores, test_articles),reverse=True)

[(1.4, 565648),
 (1.2, 565589),
 (1.2, 563115),
 (0.9, 563643),
 (0.6, 565822),
 (0.6, 555528),
 (0.4, 564335),
 (0.4, 563938),
 (0.3, 565364),
 (0.3, 560290),
 (0.1, 552077),
 (0.0, 563787),
 (0.0, 560620),
 (0.0, 560518),
 (0.0, 559855)]